# ModOAP - Extraction des annotations de régions d'illustration dans des documents Gallica

Ce script permet de récupérer les annotations des régions représentant des illustrations dans des pages de documents historiques stockés sur Gallica.

A partir de liens ARK contenus dans un fichier excel, le script télécharge pour chaque document les pages qu'il contient, et génère un fichier d'annotation VIA (format json) contenant les régions des illustrations des documents. 

Le dossier téléchargé par ce script peut ensuite être directement utilisé pour l'entraînement d'un algorithme de détection d'illustrations. 

**Fonctionnement :** 

- Synchroniser un compte Google Drive
- Spécifier un fichier XLS dont une colonne contient des liens vers des documents Gallica (ex : https://gallica.bnf.fr/ark:/12148/bpt6k54715050)
- Spécifier l'index de la colonne contenant ces liens
- Spécifier un dossier de destination où télécharger les documents


In [ ]:
#@title ##  Préparation

#@markdown ### Synchronisation d'un Google Drive
#@markdown ##### Lancer cette cellule et cliquer sur le lien généré pour autoriser un compte Google Drive

try :
    import xmltodict
except ModuleNotFoundError :
    !pip install xmltodict
import shutil
import requests
import xmltodict
from bs4 import BeautifulSoup
from openpyxl import load_workbook
import urllib.request, urllib.error, urllib.parse
from urllib.error import HTTPError, URLError
from xml.etree import ElementTree as ET
from google.colab import drive
import os
import glob
import json

# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
    drive.mount('/content/drive/')

"""if not os.path.exists("/content/drive/MyDrive/Outils_Modoap/Scrap"):
    os.makedirs("/content/drive/MyDrive/Outils_Modoap/Scrap")    

%cd /content/drive/MyDrive/Outils_Modoap/Scrap"""

# définition des fonctions
def paginationDL(ark):
    try :
        PAGINATION_BASEURL = 'https://gallica.bnf.fr/services/Pagination?ark='
        url = "".join([PAGINATION_BASEURL, ark])
        s = requests.get(url, stream=True)
        pagination = str(BeautifulSoup(s.content,"lxml-xml"))
        paginationdic = xmltodict.parse(pagination)
        nb_pages = int(paginationdic["livre"]["structure"]["nbVueImages"])
        with open("pagination_"+str(ark), "w") as pagout :
            pagout.write(pagination)
        return paginationdic, nb_pages
    except :
        print("la pagination n'a pas été téléchargée")
        

def altoDL(ark,page, ordre, numero):
    
    OCR_BASEURL = 'https://gallica.bnf.fr/RequestDigitalElement?O='
    url = "".join([OCR_BASEURL, ark, '&E=ALTO&Deb=', str(page)])
    s = requests.get(url, stream=True)
    alto = str(BeautifulSoup(s.content,"lxml-xml"))
    nomfichier = "view_"+str(ordre)+"_num_"+str(numero)+"_alto.xml"
    with open(nomfichier, "w") as altout :
        altout.write(alto)
    if len(alto) < 40 :
        return "no", nomfichier
    else :
        return "yes", nomfichier

def nb_pages(manifeste) : 
    with open(manifeste) as f:
        dico = json.load(f)
    nb_pages = len(dico["sequences"][0]["canvases"])
    return nb_pages

def manifesteDL(ark):
    try :
        url = "https://gallica.bnf.fr/iiif/ark:/12148/"+str(ark)+"/manifest.json"
        nom_fichier = ark+"-manifest.json"
        urllib.request.urlretrieve(url, nom_fichier)
        return nom_fichier
    except :
        print("Le manifeste n'a pas pu être téléchargé")

def pageDL(ark, page, numero_page, numero_phys, region="full", size="full", rotation="0", quality="native", format="jpg"):
    IIIF_BASEURL = 'https://gallica.bnf.fr/iiif/ark:/12148/'
    url = "".join([IIIF_BASEURL, ark, '/f', str(page), '/', region, '/', size, '/', rotation, '/', quality, '.', format])
    nom_fichier = ark+"_view_"+str(ordre)+"_num_"+str(numero)+".jpg"
    try :
        urllib.request.urlretrieve(url, nom_fichier)
        return "ok"
    except (HTTPError, URLError) as erreur:
        return str(erreur.reason)        

def bnf2gall(arkbnf):
    url = "https://catalogue.bnf.fr/ark:/12148/"+str(arkbnf)
    s = requests.get(url, stream=True)
    html = BeautifulSoup(s.content,"lxml-xml")
    for link in html.findAll('a', {'class': 'exemplaire-action-visualiser'}):
        ark = link['href'].split("/")[-1]
        return ark


def page_courante(ark, page, pagination):
    ordre = pagination["livre"]["pages"]["page"][int(page-1)]["ordre"]
    numero = pagination["livre"]["pages"]["page"][int(page-1)]["numero"]
    return ordre, numero

In [ ]:
# Récupération des liens depuis le fichier xls

#@title ## 0. Préparation

#@markdown ### Entrez le chemin du fichier xls :
chemin_fichier_xls = "/content/drive/MyDrive/Outils_Modoap/Scrap/excels/test2.xlsx" #@param {type:"string"}

#@markdown Possibilité de copier/coller le chemin depuis la fenêtre de gauche : onglet "Fichiers" -> clic droit sur un dossier -> "Copier le chemin"

#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/fichiers/

#@markdown ---

#@markdown ### Entrez l'indice de la colonne contenant les liens ARK :
colonne_ark = "A" #@param {type:"string"}
#@markdown Exemple d'indice : A

#@markdown ---


#@markdown ### Entrez le répertoire de destination où télécharger les documents :
chemin_destination = "/content/drive/MyDrive/Outils_Modoap/Scrap/test3" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/datasets/

#@markdown ---

chemin_classeur = chemin_fichier_xls

try :
  if not os.path.exists(chemin_destination):
      os.makedirs(chemin_destination)
except :
  print("Le chemin de destination est incorrect")

arks = []

# Chargement du fichier xls
try :
  classeur= load_workbook(chemin_classeur)
except :
  print("Le fichier xls n'a pas été chargé correctement")

# Récupération des liens ARK
for onglet in classeur.sheetnames:
    onglet_courant = classeur[onglet]
    colonne = onglet_courant[colonne_ark]
    for cellule in colonne :
        if str(cellule.value).startswith("http") or str(cellule.value).startswith("ark"):
            arks.append(str(cellule.value))

arks = set(arks)
print("{0} liens distincts récupérés dans {1} onglets".format(len(arks), len(classeur.sheetnames)))

# Tri des liens Gallica
arks_gallica = []
arks_autres_serveurs = []

for ark in arks :
  
  if ark.endswith("/") :
    ark = ark[:-1]
  ark2 = ark.split("/")[-1].strip()
  print("ark2 : ", ark2)
  if "?" in ark2 :
    ark2 = str(ark2.split("?")[0])
  if ark2.endswith("item") :
    ark2 = ark.split("/")[-2]
  if ark2.startswith("b") :
    arks_gallica.append(ark2)
  elif ark2.startswith("c") :
    ark2 = bnf2gall(ark2)
    arks_gallica.append(ark2)
  else : 
    arks_autres_serveurs.append(ark2)

print("{0} liens Gallica et {1} sur d'autres serveurs".format(len(arks_gallica), len(arks_autres_serveurs)))

%cd $chemin_destination


erreurs_500 = []
sans_ocr = []

if len(arks_gallica) > 0 :
  for ark in arks_gallica :
    fichiers_telecharges = [fich.split("/")[-1] for fich in glob.glob(os.path.join(chemin_destination,"*.*"))]
    
    print("Ark : ", ark)
   
    # Téléchargement du fichier de pagination
    pagination, nb_pages = paginationDL(ark)
    print("Nombre de pages : ", str(nb_pages))
    for page in range(nb_pages) :
      page += 1
      print("Page {0}/{1}".format(str(page), str(nb_pages)))
      ordre, numero = page_courante(ark, page, pagination)
    
    # Téléchargement du fichier alto
      if "view_"+str(ordre)+"_num_"+str(numero)+"_alto.xml" in fichiers_telecharges :
        print("Fichier Alto déjà téléchargé")
      else :
        reponse_alto, nomfichier = altoDL(ark,page, ordre, numero)
        if reponse_alto =="yes" :
          # Récupération des régions
          ###################################################
          with open(nomfichier, "r") as fichalto :
            alto = fichalto.read()
    
          altodic = xmltodict.parse(alto)
          illustrations = []

          try :
            element = altodic["alto"]["Layout"]["Page"]["PrintSpace"]["Illustration"]
            if isinstance(element, list) :
              for e in element :
                illustrations.append(e)
            else : 
                illustrations.append(element)
          except KeyError :
            pass

          try :
            if isinstance(altodic["alto"]["Layout"]["Page"]["PrintSpace"]["ComposedBlock"], dict):


              element = altodic["alto"]["Layout"]["Page"]["PrintSpace"]["ComposedBlock"]["Illustration"]
              if isinstance(element, list) :
                for e in element :
                  illustrations.append(e)
              else : 
                  illustrations.append(element)
      
            elif isinstance(altodic["alto"]["Layout"]["Page"]["PrintSpace"]["ComposedBlock"], list):
              for element in altodic["alto"]["Layout"]["Page"]["PrintSpace"]["ComposedBlock"] :
                if "Illustration" in element :
                  if isinstance(element, list) :
                    for e in element :
                      illustrations.append(e)
                  else : 
                    illustrations.append(element)

          except : pass
          liste_regions = []
          for illustration in illustrations : 
            hpos = int(illustration["@HPOS"])
            vpos = int(illustration["@VPOS"])
            width = int(illustration["@WIDTH"])
            height = int(illustration["@HEIGHT"])
            dico_reg_att = {"type" : "illustration"}
            dico_shape_att = {"all_points_x" : [hpos, hpos+width, hpos+width, hpos] , "all_points_y" : [vpos, vpos, vpos+height, vpos+height], "name" : "polygon"}
            dico_region = {"region_attributes" : dico_reg_att , "shape_attributes" : dico_shape_att }
            liste_regions.append(dico_region)

          dicofichier = {"filename" : ark+"_view_"+str(ordre)+"_num_"+str(numero)+".jpg", "regions" : liste_regions }
          dico_illustrations_fichier = {ark+"_view_"+str(ordre)+"_num_"+str(numero)+".jpg" : dicofichier}

          try : 
            with open("via_annotations.json", "r") as jjson :
              temp = jjson.read()
              dicototal = json.loads(temp)
          except FileNotFoundError:
            dicototal = {}
          dicototal[nomfichier] = dicofichier
          with open("via_annotations.json", "w") as jjson :
            json.dump(dicototal, jjson)

        if reponse_alto == "no" :
          sans_ocr.append(ark)
          print("ocr indisponible")
        ###################################################
  

    # Téléchargement de l'image de la page'
      
      if ark+"_view_"+str(ordre)+"_num_"+str(numero)+".jpg" in fichiers_telecharges :
        print("Fichier Image déjà téléchargé")
      else :
        reponse_doc = pageDL(ark, page, ordre, numero)
        if reponse_doc == "ok" :
          pass
        if reponse_doc == "500" :
          erreurs_500.append(ark)
          print("erreur 500")
          break
        else : pass 
        
for fichier in glob.glob(os.path.join(chemin_destination,"*.xml")) :
  os.remove(fichier)
for fichier in glob.glob(os.path.join(chemin_destination,"*")) :
  if "pagination" in fichier : 
    os.remove(fichier)

erreurs_500 = set(erreurs_500)
with open("erreurs_500.txt", "w") as err :
  err.write("=============================")
  err.write("Erreurs 500 : ")
  err.write("=============================")
  for ark in erreurs_500 :
    err.write(ark)
    err.write("\n")
  err.write("=============================")
  err.write("Documents non-océrisés : ")
  err.write("=============================")
  for ocrless in sans_ocr :
    err.write(ocrless)
  
  
print("Documents non téléchargés (erreur 500) : {0}".format(len(erreurs_500)))
print("Ces documents sont listés dans le fichier erreurs_500.txt")